In [ ]:
import matplotlib.pyplot as plt
import random
import numpy as np
import copy
import seaborn as sns
sns.set_theme()

In [ ]:
random.seed = 42

In [ ]:
k = 4
N = 400

x = [random.random() for _ in range(N)]
y = [random.random() for _ in range(N)]
dots = [(x[i], y[i]) for i in range(N)]

In [ ]:
def distance_manhattan(a, b):
    return abs(a[0]-b[0]) + abs(a[1]-b[1])


def distance_chebishev(a, b):
    return max(abs(a[0]-b[0]), abs(a[1]-b[1]))


def distance_euclidian(a, b):
    return np.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2)


def init_max(distance):
    distances = []
    centers = [np.mean(x), np.mean(y), -1]
    for i, d in enumerate(dots):
        distances.append((distance(dots[centers[-1]], d), i))
    centers = [sorted(distances)[-1][1]]
    for _ in range(k-1):
        distances = []
        for i, d in enumerate(dots):
            distances.append((sum([distance(dots[c], d) for c in centers]), i))
        centers.append(sorted(distances)[-1][1])
    return sorted(centers)


def init_defmax(distance):
    max_ = [0 for _ in range(k+1)]
    for i1 in range(N):
        for i2 in range(i1+1, N):
            for i3 in range(i2+1, N):
                for i4 in range(i3+1, N):
                    print(i1, i2, i3, i4)
                    d = distance(dots[i1], dots[i2]) + distance(dots[i1], dots[i3]) + distance(dots[i1], dots[i4]) + distance(dots[i2], dots[i3]) + distance(dots[i2], dots[i4]) + distance(dots[i3], dots[i4])
                    if d > max_[0]:
                        max_[0] = d
                        max_[1] = i1
                        max_[2] = i2
                        max_[3] = i3
                        max_[4] = i4
    return max_[1::]
                        
                        
def init_random(distance):
    centers = set()
    while len(centers) < k:
        centers.add(random.randint(0, N-1))
    return sorted(list(centers))


def kmeans(init_center, distance):
    centers = [dots[c] for c in init_center(distance)]
    condition = True
    iterations = 0
    while condition:
        prev_centers = copy.copy(centers)
        iterations += 1
        clusters = [[] for _ in range(k)]
        for i, d in enumerate(dots):
            j = min([(distance(d, centers[j]), j) for j in range(k)])[1]
            clusters[j].append(i)
        for index, cluster in enumerate(clusters):
            cluster_center = sum([dots[j][0] for j in cluster])/len(cluster), sum([dots[j][1] for j in cluster])/len(cluster)
            centers[index] = cluster_center
        condition = ((np.matrix(prev_centers) != np.matrix(centers))).any()
    return clusters, iterations, centers


def plot_kmeans(init, distance):
    clusters, iterations, centers = kmeans(init, distance)
    colors = ["r", "g", "b", "y", "c", "m", "k", "w"][0:k]
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(1, 1, 1)
    for data, color in zip(clusters, colors):
        x, y = [dots[i][0] for i in data], [dots[i][1] for i in data]
        ax.scatter(x, y, alpha=0.7, c=color, edgecolors='none', s=60)
    x, y = [r[0] for r in centers], [r[1] for r in centers]
    ax.scatter(x, y, alpha=1, c=colors, edgecolors='none', s=300)
    plt.title(f'{iterations} итераций, {N} точек')
    plt.show()

In [ ]:
plot_kmeans(init_random, distance_euclidian)

In [ ]:
plot_kmeans(init_max, distance_euclidian)

In [ ]:
N = 400
circle_r = 1
circle_x = 0
circle_y = 0
alpha = [2 * np.pi * random.random() for _ in range(N)]
r = [circle_r * np.sqrt(random.random()) for _ in range(N)]
x = [r[i] * np.cos(alpha[i]) + circle_x for i in range(N)]
y = [r[i] * np.sin(alpha[i]) + circle_y for i in range(N)]
dots = [(x[i], y[i]) for i in range(N)]

In [ ]:
plot_kmeans(init_max, distance_chebishev)

In [ ]:
plot_kmeans(init_max, distance_euclidian)

In [ ]:
plot_kmeans(init_max, distance_manhattan)

In [ ]:
res = [0, 0, 0]
N = 40000
for i in range(1):
    alpha = [2 * np.pi * random.random() for _ in range(N)]
    r = [circle_r * np.sqrt(random.random()) for _ in range(N)]
    x = [r[i] * np.cos(alpha[i]) + circle_x for i in range(N)]
    y = [r[i] * np.sin(alpha[i]) + circle_y for i in range(N)]
    dots = [(x[i], y[i]) for i in range(N)]
    res[0] += kmeans(init_max, distance_chebishev)[1]
    res[1] += kmeans(init_max, distance_euclidian)[1]
    res[2] += kmeans(init_max, distance_manhattan)[1]

res